In [2]:
import deepxde as dde
from deepxde.backend import tf
import numpy as np
from matplotlib import pyplot as plt

In [9]:
k = 0.4                                                                     # constant
L = 1
n = 1

geom = dde.geometry.Interval(0,L)
timedomain = dde.geometry.TimeDomain(0,n)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)                     # Making a domain of space and time

ic = dde.icbc.IC(geomtime, lambda x: np.sin(n* np.pi * x[:, 0:1]/L), lambda _, on_initial: on_initial) # x[:,0:1] will return all row and first column only -> dimension will be (m, 1). DON'T USE x[:,0] TO AVOID GETTING THE FINAL DIM -> (m,)

def double_first_column(input_array):
    return 2 * input_array[:, 0:1]                                          # x[:,0:1] will return all row and first column only -> dimension will be (m, 1). DON'T USE x[:,0] TO AVOID GETTING THE FINAL DIM -> (m,)

bc = dde.icbc.DirichletBC(geomtime, lambda input_array: double_first_column(input_array), lambda _, on_boundary: on_boundary)

def pde(comp,u):
    du_t = dde.grad.jacobian(u, comp, i=0, j=1)                             # Taking a jacobian of u (temperature/ x-directional velocity = scalar) over computational space and time. i,j are the value you choose to get the right value from the jacobian matrix.
    du_xx = dde.grad.hessian(u, comp, i=0, j=0)                             # Same way hessian of u and i, j value to choose the right value from the matrix. (i = row, j = column)
    return du_t - k * du_xx


data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc,ic],
    num_domain= 2540,
    num_boundary= 80,
    num_initial= 160,
    num_test= 2540
)

net = dde.nn.FNN([2] + [20]*3 + [1], "tanh", "Glorot normal")

In [10]:
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)

Compiling model...
'compile' took 1.717535 s



In [14]:
model.restore("model.ckpt-15000.pt", verbose=1)

Restoring model from model.ckpt-15000.pt ...



In [12]:
# To see the training and testing loss
dde.saveplot(losshistory, train_state, issave= True, isplot= True)

NameError: name 'losshistory' is not defined